# 空间转录组学任务考核文档 20230719
* 本文档分为两个任务，两任务间有关联性


* 如需要，可以联系提供服务器空间和计算支持

## 任务一 搜集空间转录组数据

* 搜集1-2个单细胞精度的空间转录组数据，需要标注空间位置和细胞类型，例如：Spatiotemporal transcriptomic atlas of mouse organogenesis using DNA nanoball-patterned arrays；


* 搜集1-2个空间转录组和单细胞转录组联合分析数据，根据单细胞数据注释细胞类型，利用映射算法标注单细胞的细胞空间位置；


* 进行转录组数据的清洗、标准化处理，请注意不同测序方法数据的处理方式区别，例如UMI和非UMI的数据处理不同。


## 任务二 生物假设问题的验证

* 利用任务一已处理的2-4个数据集（带有空间位置、细胞类型注释的单细胞表达矩阵）

* #### 生物假设问题：对于高潜力相分离蛋白，在同一类型细胞之间，随着细胞空间距离增大，是否基因表达的异质性（表达噪音）会增大？


* 两类高潜力相分离蛋白：1）已知相分离蛋白列表： 20221205-phasepdbv2_1_llps.xlsx； 2）预测分数top 10%蛋白：20220824-human_LLPS_score.csv，包括以下指标：'rnk_PLAAC', 'rnk_PScore', 'rnk_SaPS', 'rnk_PdPS', 'rnk_SaPS-10', 'rnk_PdPS-10', 'rnk_catGRANULE', 'rnk_DeepPhase'，指标均为计算好的百分比指标。


* 提供各基因不同数据库的命名转换表格：uniprot_gene_ensembl.csv


* 文件下载地址：链接：https://pan.baidu.com/s/1eXyU-MGDFQYnqkrw9o8x2g?pwd=6mvp 提取码：6mvp 

In [ ]:
# 细胞表达异质性计算方式示例：
# 两种指标：rCVDM和rSTDDM(滚动中位数矫正差异)

tls = list(set([file.split('_')[0] for file in os.listdir('/mnt/md1400/yl/SC-PS/data/HPA/lognorm/%s/'%cway)]))

for tissue in tls:
    tsta_data = pd.DataFrame()
    for file in os.listdir('/mnt/md1400/yl/SC-PS/data/HPA/lognorm/%s/'%cway):
        ct = '_'.join(file.split('_')[:-3])
        if(ct.split('_')[0] == tissue):
            print('\r%s'%ct, end = '')
            ct_exdata = pd.read_csv('/mnt/md1400/yl/SC-PS/data/HPA/lognorm/%s/%s_tpm_norm_log.csv'%(cway, ct), index_col = 0)
            ct_exdata = ct_exdata.groupby(ct_exdata.index).first()

            sta_data = pd.DataFrame()
            sta_data['%s_mean'%ct] = ct_exdata.apply(lambda x: np.mean(list(x)), axis = 1)
            sta_data['%s_rmean'%ct] = sta_data['%s_mean'%ct].rank(ascending = 1, method = 'dense')
            sta_data['%s_rmean'%ct] = sta_data['%s_rmean'%ct]/sta_data['%s_rmean'%ct].max()

            sta_data['%s_median'%ct] = ct_exdata.apply(lambda x: np.median(list(x)), axis = 1)
            sta_data['%s_rmedian'%ct] = sta_data['%s_median'%ct].rank(ascending = 1, method = 'dense')
            sta_data['%s_rmedian'%ct] = sta_data['%s_rmedian'%ct]/sta_data['%s_rmedian'%ct].max()

            sta_data['%s_STD'%ct] = ct_exdata.apply(lambda x: np.std(list(x), ddof = 1), axis = 1)
            sta_data['%s_rSTD'%ct] = sta_data['%s_STD'%ct].rank(ascending = 1, method = 'dense')
            sta_data['%s_rSTD'%ct] = sta_data['%s_rSTD'%ct]/sta_data['%s_rSTD'%ct].max()

            sta_data['%s_CV'%ct] = sta_data['%s_STD'%ct]/sta_data['%s_mean'%ct]
            sta_data['%s_rCV'%ct] = sta_data['%s_CV'%ct].rank(ascending = 1, method = 'dense')
            sta_data['%s_rCV'%ct] = sta_data['%s_rCV'%ct]/sta_data['%s_rCV'%ct].max()

            sta_data = sta_data.sort_values('%s_mean'%ct)
            sta_data['%s_rollmSTD'%ct] = sta_data['%s_STD'%ct].rolling(window = 50, center = False, min_periods = 25).median()
            sta_data['%s_STDDM'%ct] = sta_data['%s_STD'%ct] - sta_data['%s_rollmSTD'%ct]
            sta_data['%s_rSTDDM'%ct] = sta_data['%s_STDDM'%ct].rank(ascending = 1, method = 'dense')
            sta_data['%s_rSTDDM'%ct] = sta_data['%s_rSTDDM'%ct]/sta_data['%s_rSTDDM'%ct].max()

            sta_data['%s_rollmCV'%ct] = sta_data['%s_CV'%ct].rolling(window = 50, center = False, min_periods = 25).median()
            sta_data['%s_CVDM'%ct] = sta_data['%s_CV'%ct] - sta_data['%s_rollmCV'%ct]
            sta_data['%s_rCVDM'%ct] = sta_data['%s_CVDM'%ct].rank(ascending = 1, method = 'dense')
            sta_data['%s_rCVDM'%ct] = sta_data['%s_rCVDM'%ct]/sta_data['%s_rCVDM'%ct].max()

            sta_data['%s_p0.1'%ct] = ct_exdata.apply(lambda x: len([y for y in list(x) if \
                                                           y > np.log(0.1 + 1)]) / len(x), axis = 1)
            sta_data['%s_num'%ct] = len(ct_exdata.columns)
            tsta_data = pd.concat([tsta_data, sta_data], axis = 1)

    tsta_data.index.name = 'Gene'
    tsta_data.to_csv('out/HPA-data/20221102-HPA-%s-%s-statistic-result.csv'%(tissue, cway), index = True)